In [0]:
import torch
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
from torch import nn,optim
from torch.utils.data import DataLoader
from torchvision import datasets,transforms
import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [0]:
train_transform = transforms.Compose([transforms.RandomHorizontalFlip(), transforms.ToTensor()])
test_transform = transforms.Compose([transforms.ToTensor()])

train_dataset=datasets.CIFAR10(root='./data1',train=True,transform=data_tf,download=True)
test_dataset=datasets.CIFAR10(root='./data1',train=False,transform=data_tf)
#使用内置的函数导入数据集
train_loader=DataLoader(train_dataset,batch_size=4,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=4,shuffle=False)


Files already downloaded and verified


In [0]:
class MyAlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(MyAlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(64, 192, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 2 * 2, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 2 * 2)
        x = self.classifier(x)
        return x
  
  


In [0]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [0]:
import torch
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
 
from torch import nn,optim
from torch.utils.data import DataLoader
from torchvision import datasets,transforms
#定义一些超参数

learning_rate=0.1
num_epoches=200


model=MyAlexNet(10)
if torch.cuda.is_available():
    model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=learning_rate, momentum=0.8)
#optimizer = optim.Adam(model.parameters(), lr = learning_rate)
n_epochs=2
for epoch in range(n_epochs):
    total=0
    running_loss=0.0
    running_correct=0
    print("epoch {}/{}".format(epoch,n_epochs))
    print("-"*10)
    for i, data in enumerate(train_loader, 0):
        img,label=data
        #img=img.view(img.size(0),-1)
        img = Variable(img)
        if torch.cuda.is_available():
            img=img.cuda()
            label=label.cuda()
        else:
            img=Variable(img)
            label=Variable(label)
        out=model(img)#得到前向传播的结果
        loss=criterion(out,label)#得到损失函数
        print_loss=loss.data.item()
        optimizer.zero_grad()#归0梯度
        loss.backward()#反向传播
        optimizer.step()#优化
        running_loss+=loss.item()
        epoch+=1
        if i % 2000 == 1999:    # 2000 ミニバッチ毎に表示する
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0
    _, predicted = torch.max(out.data, 1)
    total += label.size(0)
    running_correct += (predicted == label).sum()
    print('Accuracy of the network on the 10000 test images: %d %%' % (100 * running_correct / total))



epoch 0/2
----------
[2001,  2000] loss: 2.332
[4001,  4000] loss: 2.332
[6001,  6000] loss: 2.330
[8001,  8000] loss: 2.330
[10001, 10000] loss: 2.331
[12001, 12000] loss: 2.331
Accuracy of the network on the 10000 test images: 25 %
epoch 1/2
----------
[2002,  2000] loss: 2.327
[4002,  4000] loss: 2.330
[6002,  6000] loss: 2.332
[8002,  8000] loss: 2.329
[10002, 10000] loss: 2.333
[12002, 12000] loss: 2.331
Accuracy of the network on the 10000 test images: 25 %
